# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,278.57,97,100,13.78,TF,1720634366
1,1,ilulissat,69.2167,-51.1000,276.39,87,96,2.58,GL,1720634367
2,2,somerset east,-32.7217,25.5880,286.16,59,54,4.84,ZA,1720634368
3,3,waitangi,-43.9535,-176.5597,281.38,91,100,5.36,NZ,1720634370
4,4,blackmans bay,-43.0167,147.3167,281.22,96,46,3.04,AU,1720634371


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    fill_alpha = 0.5,
    title = "Cities in City_data_df",
    xlabel = "Latitude",
    ylabel = "Longitude")

# Display the map
map1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Filter the city_data_df DataFrame for my family's ideal weather for vacationing
ideal_weather_df = city_data_df.loc[
    (city_data_df["Humidity"] <= 70) &
    (city_data_df["Max Temp"] >= 273) &
    (city_data_df["Max Temp"] <= 297) &
    (city_data_df["Wind Speed"] <= 10) &
    (city_data_df["Cloudiness"] <= 50) 
    ]

# Drop any rows with null values
ideal_weather_df.dropna().reset_index(drop=True)

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,egilsstadir,65.2653,-14.3948,294.10,37,0,4.63,IS,1720634372
27,27,mariehamn,60.0973,19.9348,292.07,59,0,5.14,AX,1720634397
60,60,college,64.8569,-147.8028,292.09,66,40,2.57,US,1720634436
75,75,labytnangi,66.6572,66.4183,294.27,64,47,2.79,RU,1720634452
76,76,extrema,-22.8547,-46.3183,296.37,54,47,4.03,BR,1720634453
...,...,...,...,...,...,...,...,...,...,...
539,539,happy valley-goose bay,53.3168,-60.3315,293.93,53,41,3.63,CA,1720634995
546,546,malaya vishera,58.8454,32.2222,291.13,61,14,1.36,RU,1720635003
548,548,kununurra,-15.7667,128.7333,292.14,68,22,3.60,AU,1720634835
564,564,taltal,-25.4000,-70.4833,286.77,68,47,3.04,CL,1720635026


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,egilsstadir,IS,65.2653,-14.3948,37,
27,mariehamn,AX,60.0973,19.9348,59,
60,college,US,64.8569,-147.8028,66,
75,labytnangi,RU,66.6572,66.4183,64,
76,extrema,BR,-22.8547,-46.3183,54,
...,...,...,...,...,...,...
539,happy valley-goose bay,CA,53.3168,-60.3315,53,
546,malaya vishera,RU,58.8454,32.2222,61,
548,kununurra,AU,-15.7667,128.7333,68,
564,taltal,CL,-25.4000,-70.4833,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
egilsstadir - nearest hotel: Valaskjálf
mariehamn - nearest hotel: Hotell Esplanad
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
labytnangi - nearest hotel: Семь лиственниц
extrema - nearest hotel: Hotel San Marco
badger - nearest hotel: Clarion Hotel & Suites
patacamaya - nearest hotel: Hostal Viña del Mar
namsos - nearest hotel: Tinos hotell
puerto natales - nearest hotel: Dorotea Patagonia Hostel
kabwe - nearest hotel: Tuskers Hotel
bilibino - nearest hotel: No hotel found
necochea - nearest hotel: Cabañas Río Quequén
udachny - nearest hotel: Вилюй
lebu - nearest hotel: No hotel found
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
port mathurin - nearest hotel: Escale Vacances
tres arroyos - nearest hotel: Andrea Hotel
castelli - nearest hotel: Hotel Clarice
tralee - nearest hotel: The Ashe Hotel
kharp - nearest hotel: Собь
karratha - nearest hotel: Karratha International Hotel
ola - nearest hotel: No hotel found
formosa - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
5,egilsstadir,IS,65.2653,-14.3948,37,Valaskjálf
27,mariehamn,AX,60.0973,19.9348,59,Hotell Esplanad
60,college,US,64.8569,-147.8028,66,BEST WESTERN PLUS Pioneer Park Inn
75,labytnangi,RU,66.6572,66.4183,64,Семь лиственниц
76,extrema,BR,-22.8547,-46.3183,54,Hotel San Marco
...,...,...,...,...,...,...
539,happy valley-goose bay,CA,53.3168,-60.3315,53,Hotel North Two
546,malaya vishera,RU,58.8454,32.2222,61,No hotel found
548,kununurra,AU,-15.7667,128.7333,68,No hotel found
564,taltal,CL,-25.4000,-70.4833,68,Hostería Taltal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    fill_alpha = 0.5,
    title = "My Family's Ideal Cities to Vacation",
    xlabel = "Latitude",
    ylabel = "Longitude",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)